In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from pandas import json_normalize
! pip install folium==0.5.0
import folium # plotting library
print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'SXVSRVQ0EG2SYNDENYWUZU1XIZ1FQU3NDBKWE1FECIAP0LM4' # your Foursquare ID
CLIENT_SECRET = 'NHTLTQC5QLK5YJKA4WFOTV4AEKABYBVJFAFJ4ZJVO3RT41N1' # your Foursquare Secret
ACCESS_TOKEN = 'E15002NNGJ30TH4WLZSD3TXKQE1H3X1DQC3E5J431LKHAQTL' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SXVSRVQ0EG2SYNDENYWUZU1XIZ1FQU3NDBKWE1FECIAP0LM4
CLIENT_SECRET:NHTLTQC5QLK5YJKA4WFOTV4AEKABYBVJFAFJ4ZJVO3RT41N1


<h3>we choose the center of Berlin as our location where we start our tour<h3/>

In [3]:
address = 'Panoramastraße 1A, 10178 Berlin, Berlin'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

52.520848 13.4084747


<h3>then we want to search for a nice coffee shop <h3/>

In [4]:
search_query = 'Coffee'
radius = 2000
print(search_query + ' .... OK!')

Coffee .... OK!


<h3>we call the foursquare API <h3/>

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION,search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=SXVSRVQ0EG2SYNDENYWUZU1XIZ1FQU3NDBKWE1FECIAP0LM4&client_secret=NHTLTQC5QLK5YJKA4WFOTV4AEKABYBVJFAFJ4ZJVO3RT41N1&ll=52.520848,13.4084747&oauth_token=E15002NNGJ30TH4WLZSD3TXKQE1H3X1DQC3E5J431LKHAQTL&v=20180604&query=Coffee&radius=2000&limit=30'

In [6]:
results = requests.get(url).json()
#results

In [7]:
#assign relevant part of JSON to park
park = results['response']['venues']
# tranform venues into a dataframe
dataframe = json_normalize(park)

dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,514453f5e4b071f0e0e0a123,Coffee Fellows,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1628082590,False,Rosa-Luxemburg-Str. 2,52.522656,13.409790,"[{'label': 'display', 'lat': 52.52265601136413...",220,10178,DE,Mitte,Berlin,Berlin,Deutschland,"[Rosa-Luxemburg-Str. 2, 10178 Berlin]",NaN
1,5d31865fd1b9890007ade245,T2 Breakfast & Coffee,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1628082590,False,Litfassplatz 3,52.521933,13.403187,"[{'label': 'display', 'lat': 52.521933, 'lng':...",377,NaN,DE,NaN,Berlin,Berlin,Deutschland,"[Litfassplatz 3, Berlin]",NaN
2,4ba898d4f964a520c4e139e3,Impala Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1628082590,False,Schönhauser Allee 173,52.532549,13.412092,"[{'label': 'display', 'lat': 52.532549, 'lng':...",1325,10119,DE,NaN,Berlin,Berlin,Deutschland,"[Schönhauser Allee 173 (Schwedter Str.), 10119...",Schwedter Str.
3,4bf61d1194b2a593a203adee,snack'n coffee,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1628082590,False,Alexanderplatz,52.521005,13.411965,"[{'label': 'display', 'lat': 52.52100527429267...",237,NaN,DE,NaN,Berlin,Berlin,Deutschland,"[Alexanderplatz, Berlin]",NaN
4,534926fa11d2adce9850f45b,Coffee Room,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1628082590,False,Saarbrücker Str. 30,52.529816,13.415447,"[{'label': 'display', 'lat': 52.52981574063849...",1104,10405,DE,NaN,Berlin,Berlin,Deutschland,"[Saarbrücker Str. 30, 10405 Berlin]",NaN


<h3>we clean and filter our data <h3/>

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
            categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,Coffee Fellows,Coffee Shop,Rosa-Luxemburg-Str. 2,52.522656,13.409790,"[{'label': 'display', 'lat': 52.52265601136413...",220,10178,DE,Mitte,Berlin,Berlin,Deutschland,"[Rosa-Luxemburg-Str. 2, 10178 Berlin]",NaN,514453f5e4b071f0e0e0a123
1,T2 Breakfast & Coffee,Breakfast Spot,Litfassplatz 3,52.521933,13.403187,"[{'label': 'display', 'lat': 52.521933, 'lng':...",377,NaN,DE,NaN,Berlin,Berlin,Deutschland,"[Litfassplatz 3, Berlin]",NaN,5d31865fd1b9890007ade245
2,Impala Coffee,Coffee Shop,Schönhauser Allee 173,52.532549,13.412092,"[{'label': 'display', 'lat': 52.532549, 'lng':...",1325,10119,DE,NaN,Berlin,Berlin,Deutschland,"[Schönhauser Allee 173 (Schwedter Str.), 10119...",Schwedter Str.,4ba898d4f964a520c4e139e3
3,snack'n coffee,Bakery,Alexanderplatz,52.521005,13.411965,"[{'label': 'display', 'lat': 52.52100527429267...",237,NaN,DE,NaN,Berlin,Berlin,Deutschland,"[Alexanderplatz, Berlin]",NaN,4bf61d1194b2a593a203adee
4,Coffee Room,Coffee Shop,Saarbrücker Str. 30,52.529816,13.415447,"[{'label': 'display', 'lat': 52.52981574063849...",1104,10405,DE,NaN,Berlin,Berlin,Deutschland,"[Saarbrücker Str. 30, 10405 Berlin]",NaN,534926fa11d2adce9850f45b
5,Balzac Coffee,Coffee Shop,Karl-Liebknecht-Str. 5,52.519681,13.403687,"[{'label': 'display', 'lat': 52.51968106379066...",349,10178,DE,NaN,Berlin,Berlin,Deutschland,"[Karl-Liebknecht-Str. 5 (Spandauer Str.), 1017...",Spandauer Str.,4b0e18d0f964a520f05423e3
6,Bonanza Coffee,Coffee Shop,Jägerstr. 58,52.514034,13.390665,"[{'label': 'display', 'lat': 52.514034, 'lng':...",1425,10117,DE,NaN,Berlin,Berlin,Deutschland,"[Jägerstr. 58, 10117 Berlin]",NaN,5eca85b8f2870e000801509b
7,Bonanza Coffee,Coffee Shop,Oderberger Str. 35,52.539993,13.405530,"[{'label': 'display', 'lat': 52.53999265350801...",2140,10435,DE,NaN,Berlin,Berlin,Deutschland,"[Oderberger Str. 35, 10435 Berlin]",NaN,4af28ef8f964a520eae721e3
8,AMT Coffee,Coffee Shop,Hackescher Markt,52.523158,13.403127,"[{'label': 'display', 'lat': 52.52315836250000...",444,10178,DE,NaN,Berlin,Berlin,Deutschland,"[Hackescher Markt (S-Bahnbögen), 10178 Berlin]",S-Bahnbögen,4b11338ff964a520d57823e3
9,the coffee shop,Café,Werderscher Markt 1,52.515441,13.397738,"[{'label': 'display', 'lat': 52.51544126096473...",944,10117,DE,NaN,Berlin,Berlin,Deutschland,"[Werderscher Markt 1 (im Auswärtigen Amt), 101...",im Auswärtigen Amt,4e4a6397c65bab12516c3dd6


<h3>then we use the Folium to visualize our map<h3/>

<h3>we are interrested in one of the coffee shops and we want to see the rates <h3/>

In [26]:
venue_id = '514453f5e4b071f0e0e0a123' # ID of coffee
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id,CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/514453f5e4b071f0e0e0a123?client_id=SXVSRVQ0EG2SYNDENYWUZU1XIZ1FQU3NDBKWE1FECIAP0LM4&client_secret=NHTLTQC5QLK5YJKA4WFOTV4AEKABYBVJFAFJ4ZJVO3RT41N1&oauth_token=E15002NNGJ30TH4WLZSD3TXKQE1H3X1DQC3E5J431LKHAQTL&v=20180604'

In [27]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes', 'bestPhoto', 'colors'])


{'id': '514453f5e4b071f0e0e0a123',
 'name': 'Coffee Fellows',
 'contact': {'phone': '03027891945',
  'formattedPhone': '030 27891945',
  'facebook': '313849360475',
  'facebookName': 'Coffee Fellows'},
 'location': {'address': 'Rosa-Luxemburg-Str. 2',
  'lat': 52.52265601136413,
  'lng': 13.409790005427109,
  'labeledLatLngs': [{'label': 'display',
    'lat': 52.52265601136413,
    'lng': 13.409790005427109}],
  'postalCode': '10178',
  'cc': 'DE',
  'neighborhood': 'Mitte',
  'city': 'Berlin',
  'state': 'Berlin',
  'country': 'Deutschland',
  'formattedAddress': ['Rosa-Luxemburg-Str. 2', '10178 Berlin']},
 'canonicalUrl': 'https://foursquare.com/v/coffee-fellows/514453f5e4b071f0e0e0a123',
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'t

In [ ]:
#to see the general rates
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

5.7


<h3>to see the tips<h3/>

In [ ]:
result['response']['venue']['tips']['count']

87

In [ ]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(venue_id,CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION, limit)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '610a94a6d41db438494872b6'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'tips': {'count': 55,
   'items': [{'id': '55f34b24498e1b1bb399cddd',
     'createdAt': 1442007844,
     'text': 'This place was amazing! I wish I found it before that last day of my visit to Berlin. Excellent place for breakfast, great range of options (plenty of vegetarian options too), awesome friendly staff!!',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/55f34b24498e1b1bb399cddd',
     'likes': {'count': 0, 'groups': []},
     'like': False,
     'logView': True,
     'agreeCount': 1,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '126697992',
      'firstName': 'Stan',
      'lastName': 'Darsh',
      'gender': 'none',
      'countryCode': 'AU',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/blank_boy.png',
       'default': True}},
     'au

In [ ]:
tips = results['response']['tips']['items']
tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'likes', 'like', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

<h3>and here we can read the top tip for the coffee shop<h3/>

In [33]:
pd.set_option('display.max_colwidth', None)
tips_df = json_normalize(tips) # json normalize tips
# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]
# display tips
tips_filtered.reindex()

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,"This place was amazing! I wish I found it before that last day of my visit to Berlin. Excellent place for breakfast, great range of options (plenty of vegetarian options too), awesome friendly staff!!",1,0,55f34b24498e1b1bb399cddd,Stan,Darsh,126697992


<h3>and after drinking our coffee we are interrested in the area of the coffee shop and we want to explore it <h3/>

<h3>and here again we clean and filter our data<h3/>

In [39]:
dataframe = json_normalize(items) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Fat Tire Bike Tours,Bike Rental / Bike Share,Panoramastr. 1a,52.521233,13.409110,"[{'label': 'display', 'lat': 52.521232885960515, 'lng': 13.409109598118372}]",164,10178,DE,Berlin,Berlin,Deutschland,"[Panoramastr. 1a, 10178 Berlin, Deutschland]",NaN,NaN,4bb8541f1261d13a118fe898
1,19grams,Coffee Shop,Karl-Liebknecht-Str. 13,52.522736,13.407344,"[{'label': 'display', 'lat': 52.52273621459739, 'lng': 13.407343626022339}]",165,10178,DE,Berlin,Berlin,Deutschland,"[Karl-Liebknecht-Str. 13 (Rochstr.), 10178 Berlin, Deutschland]",Rochstr.,Mitte,5b55e0a9acb00b0039fb45e3
2,MYKITA,Optical Shop,Rosa-Luxemburg-Straße 6,52.523520,13.409480,"[{'label': 'display', 'lat': 52.52352, 'lng': 13.40948}]",98,10178,DE,Berlin,Berlin,Deutschland,"[Rosa-Luxemburg-Straße 6, 10178 Berlin, Deutschland]",NaN,NaN,57e908c6498e2af8ac3f326f
3,Flying Tiger,Gift Shop,Alexanderplatz 8,52.522437,13.412612,"[{'label': 'display', 'lat': 52.52243722587935, 'lng': 13.412612279052135}]",192,10178,DE,Berlin,Berlin,Deutschland,"[Alexanderplatz 8, 10178 Berlin, Deutschland]",NaN,NaN,588875294745863ba6709dcf
4,Father Carpenter,Coffee Shop,Münzstr. 21,52.524469,13.406614,"[{'label': 'display', 'lat': 52.52446934407154, 'lng': 13.406614065170288}]",294,10178,DE,Berlin,Berlin,Deutschland,"[Münzstr. 21 (1. Hof), 10178 Berlin, Deutschland]",1. Hof,NaN,54bd1e28498eec46cfc17c24
5,Klub Kitchen,Bistro,Almstadtstr. 9-11,52.524849,13.408988,"[{'label': 'display', 'lat': 52.524849, 'lng': 13.408988}]",250,10119,DE,Berlin,Berlin,Deutschland,"[Almstadtstr. 9-11, 10119 Berlin, Deutschland]",NaN,NaN,551706fe498e237bfc874019
6,Lebensmittel in Mitte,German Restaurant,Rochstr. 2,52.524150,13.407660,"[{'label': 'display', 'lat': 52.52415016934014, 'lng': 13.407659595853865}]",220,10178,DE,Berlin,Berlin,Deutschland,"[Rochstr. 2, 10178 Berlin, Deutschland]",NaN,NaN,4b0d8651f964a520304a23e3
7,Shiori,Japanese Restaurant,Max-Beer-Straße 13,52.525501,13.408275,"[{'label': 'display', 'lat': 52.5255006388942, 'lng': 13.40827536421529}]",332,10119,DE,Berlin,Berlin,Deutschland,"[Max-Beer-Straße 13, 10119 Berlin, Deutschland]",NaN,NaN,579a410b498e5f7c187c5d91
8,dolores*,Burrito Place,Rosa-Luxemburg-Str. 7,52.523736,13.409249,"[{'label': 'display', 'lat': 52.523736, 'lng': 13.409249}]",125,10178,DE,Berlin,Berlin,Deutschland,"[Rosa-Luxemburg-Str. 7, 10178 Berlin, Deutschland]",NaN,NaN,4adcda88f964a520854921e3
9,LUSH,Cosmetics Shop,Rathausstr. 5,52.519844,13.410409,"[{'label': 'display', 'lat': 52.51984357496295, 'lng': 13.410409304434047}]",315,10178,DE,Berlin,Berlin,Deutschland,"[Rathausstr. 5, 10178 Berlin, Deutschland]",NaN,NaN,4c28ad7997d00f47212d40ea


<h1>the full work will be in week 2 , here i need just to explain the data and the project :)<h1/>